# 📑 Evaluación de PubMedCLIP en ROCO

Este notebook forma parte del proyecto de tesis sobre **medición del balance multimodal en modelos CLIP aplicados a dominios médicos**.

Modelo actual: **PubMedCLIP**  
Dataset: **ROCO (Radiology Objects in COntext)**  
Tarea: **ISA (Image-Sentence Alignment)**
---


## Clonar repositorio

In [ ]:
# 📌 Código
REPO_URL  = "https://github.com/Alberto-97sc/mmshap_medclip.git"
LOCAL_DIR = "/content/mmshap_medclip"
BRANCH    = "main"

%cd /content
import os, shutil, subprocess, sys

if not os.path.isdir(f"{LOCAL_DIR}/.git"):
    # No está clonado aún
    !git clone $REPO_URL $LOCAL_DIR
else:
    # Ya existe: actualiza a la última versión del remoto
    %cd $LOCAL_DIR
    !git fetch origin
    !git checkout $BRANCH
    !git reset --hard origin/$BRANCH
%cd $LOCAL_DIR
!git rev-parse --short HEAD

## Instalar dependencias y montar google drive

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

# === Instalar en modo editable (pyproject.toml) ===
%pip install -e /content/mmshap_medclip

In [ ]:
# 📌 Código
from google.colab import drive; drive.mount('/content/drive')

## Cargar modelos y datos

In [ ]:
# 📌 Código
CFG_PATH="/content/mmshap_medclip/configs/roco_isa_pubmedclip.yaml"

# Asegura que cfg, device, dataset y model estén listos en esta sesión
if not all(k in globals() for k in ("cfg", "device", "dataset", "model")):
    from mmshap_medclip.io_utils import load_config
    from mmshap_medclip.devices import get_device
    from mmshap_medclip.registry import build_dataset, build_model

    cfg = load_config(CFG_PATH)
    device  = get_device()
    dataset = build_dataset(cfg["dataset"])
    model   = build_model(cfg["model"], device=device)

print("OK → len(dataset) =", len(dataset), "| device =", device)

## Ejecutar SHAP en una muestra

In [ ]:
# 📌 Código
from mmshap_medclip.tasks.isa import run_isa_one

muestra = 266
sample  = dataset[muestra]
image, caption = sample['image'], sample['text']

res = run_isa_one(model, image, caption, device, explain=True, plot=True)
print(f"logit={res['logit']:.4f}  TScore={res['tscore']:.2%}  IScore={res['iscore']:.2%}")
# Si quieres la figura:
# display(res['fig'])
